## Simulación de MonteCarlo

In [1]:
import numpy as np
from numpy.linalg import cholesky

np.random.seed(77)

In [2]:
def MonteCarlo(initial_maturity, S_0, num_sims, num_assets, num_asian_dates, value_date_index, correl_matrix,risk_free_rate, vols):
    '''
  Inputs:
  -------
  * S_0 (float): initial spot prices of the assets
  * num_asian_dates (int): number of asian dates. Notice that this includes the initial date, which fixes the initial spot values.
  * value_date_index (int): index, within the array of asian dates indicating the value date. (Actual step)
  * risk_free_rate (float): risk free rate
  * num_assets (int): number of underlying assets.
  * vols (array(float)): array of indiv assets vols.
  * correl_matrix (array(float, float)): matrix of correlations
  * initial_maturity (float): maturity of the product as seen on initial spot fixing date. (Years)
  * num_sims (int): number of simulations
  Outputs:
  --------
  * rets (numpy.ndarray): simulation of the evolution of the assets. Shape (num_sims, num_asian_dates-1, num_assets)
  * payoff (numpy.ndarray): Payoff of every simulation done. Shape (num_sims, )
  * premium (numpy.float64): present value in the actual step given by value_date_index.
  '''

    # Inputs:

    # Simulation of assets up to value date:

    init_time_array = np.linspace(0, initial_maturity, num_asian_dates)

    delta_t = initial_maturity / (num_asian_dates - 1)
    num_steps = value_date_index
    num_remaining_steps = num_asian_dates - value_date_index -1

    # Independent brownians
    inc_W = np.random.normal(size=(num_assets, num_steps), scale = np.sqrt(delta_t))

    # Cholesky matrix
    m = cholesky(correl_matrix)

    # Correlated brownians
    inc_W_correl = np.matmul(m, inc_W)

    # Independent brownian motion (the axis order is done to be able to correlate them with a matrix multiplication)
    inc_W_remaining = np.random.normal(size=(num_sims, num_remaining_steps, num_assets), scale = np.sqrt(delta_t))

    # We correlate them
    inc_W_correl_remaining = np.matmul(inc_W_remaining, m.T)

    # We transpose the 3D matrix of correlated B. motion (path, asset, time step)

    inc_W_correl_remaining = inc_W_correl_remaining.transpose([0,2,1])

    aux = np.repeat(inc_W_correl[None,...],num_sims,axis=0)

    # We attach the brownians obtained from t= 0 to value date

    inc_W_correl_total = np.concatenate((aux, inc_W_correl_remaining), axis = 2)

    # We compute exponential returns

    gross_rets_total = np.exp((risk_free_rate - 0.5 *vols **2) * delta_t + vols * inc_W_correl_total)

    # We simulate the underlyings

    S_T = np.cumprod(np.concatenate((np.repeat(S_0.reshape(-1,1)[None,...],num_sims,axis=0), gross_rets_total), axis = 2), axis = 2)

    # We compute the returns

    rets = S_T[:,:,1:] / np.repeat(S_0.reshape(-1,1)[None,...],num_sims,axis=0)

    payoff = np.maximum(np.sum(np.prod(rets, axis = (2))**(1/(num_assets * (num_asian_dates-1))), axis = 1)-3,0)

    # Calculate the premium as the discounted average value

    premium = np.mean(payoff*np.exp(-risk_free_rate*(initial_maturity-init_time_array[value_date_index])))

    rets = rets.transpose([0,2,1])

    return rets, payoff, premium

In [3]:
from scipy.stats import norm

N = norm.cdf

def Black(Forward, Strike, TTM, rate, Vol, IsCall):

  '''
  Inputs:
  -------
    Forward (float): Forward value
    Strike (float): strike price
    TTM (float): time to maturity in years
    rate (float): risk free rate
    Vol (float): volatility
    IsCall (bool): True if call option, False if put option
  Outputs:
  --------
    Option premium (float)
  '''

  if TTM >0:

    d1 = (np.log(Forward/Strike) + (Vol*Vol/2)*TTM)/(Vol*np.sqrt(TTM))
    d2 = (np.log(Forward/Strike) + (- Vol*Vol/2)*TTM)/(Vol*np.sqrt(TTM))

    if IsCall:

      return (Forward*N(d1)-Strike*N(d2))*np.exp(-rate*TTM)

    else:

      return (-Forward*N(-d1)+Strike*N(-d2))*np.exp(-rate*TTM)

  else:

    if IsCall:

      return np.maximum(Forward-Strike,0)

    else:

      return np.maximum(-Forward+Strike,0)

In [4]:
def BasketGeomAsian(num_asian_dates, value_date_index, risk_free_rate, num_assets, assets_vol, assets_correl, initial_maturity, price_history, IsCall):
  '''
  Inputs:
  -------
  * num_asian_dates (int): number of asian dates. Notice that this includes the initial date, which fixes the initial spot values.
  * value_date_index (int): index, within the array of asian dates indicating the value date.
  * risk_free_rate (float): risk free rate
  * num_assets (int): number of underlying assets.
  * assets_vol (array(float)): array of indiv assets vols.
  * assets_correl (array(float, float)): matrix of correlations
  * initial_maturity (float): maturity of the product as seen on initial spot fixing date.
  * price_history (array(float, float)): history of fixings of the underlyings up to value date. Assets per row, time steps per column.
  * IsCall (bool): True if call option, False if put option
  Outputs:
  --------
  * Option price (float)
  '''

  init_time_array = np.linspace(0, initial_maturity, num_asian_dates)

  pending_times_array = init_time_array[value_date_index+1:] - init_time_array[value_date_index]

  mu = np.sum(risk_free_rate - 0.5*assets_vol*assets_vol)*np.sum(pending_times_array) / (num_assets * (num_asian_dates-1))

  diag_vol = np.diag(assets_vol.reshape(-1))

  cov_matrix = np.matmul(diag_vol, np.matmul(assets_correl, diag_vol))

  xx, yy = np.meshgrid(pending_times_array, pending_times_array, sparse=True)
  z = np.minimum(xx, yy)

  V = np.sum(cov_matrix) * np.sum(z) / (num_assets*num_assets*(num_asian_dates-1)*(num_asian_dates-1))

  Forward = np.power(np.prod(price_history[:, 1:value_date_index+1] / price_history[:,0].reshape(-1,1)),1.0/(num_assets * (num_asian_dates-1)))

  Forward *= np.power(np.prod(price_history[:,value_date_index] / price_history[:,0]), (num_asian_dates-value_date_index-1)/(num_assets * (num_asian_dates-1)))

  Forward *= np.exp(mu + 0.5 * V)

  remaining_maturity = initial_maturity - init_time_array[value_date_index]


  return Black(Forward, 1.0, remaining_maturity, risk_free_rate,np.sqrt(V / remaining_maturity), IsCall)


In [5]:
S_0 = np.array([1,1,1])
num_steps = 20
vols = np.array([0.29,0.38,0.33]).reshape(-1,1)
risk_free_rate = 0.01


TTM = initial_maturity = 5 #years
value_date_index = 5 #actual step
num_asian_dates = num_steps + 1 #Time steps + 1
init_time_array = np.linspace(0, initial_maturity, num_asian_dates)

rho = 0.1
correl_matrix = rho = np.array([
    [1, 0.275125, 0.329366],
    [0.275125, 1, 0.381295],
    [0.329366, 0.381295, 1]
]) 


num_sims = 100000

num_assets = 3 #numero de activos

rets, payoff, premium = MonteCarlo(initial_maturity, S_0, num_sims, num_assets, num_asian_dates, value_date_index, correl_matrix,risk_free_rate, vols)

In [6]:
print(rets)

[[[1.02483539 0.69452105 0.94906196]
  [1.11900687 0.74440623 1.17127183]
  [1.02057297 0.53319234 0.88871561]
  ...
  [0.63693688 0.17206475 0.26833344]
  [0.67225676 0.20161834 0.33230861]
  [0.65371429 0.1524178  0.36082416]]

 [[1.02483539 0.69452105 0.94906196]
  [1.11900687 0.74440623 1.17127183]
  [1.02057297 0.53319234 0.88871561]
  ...
  [1.72532464 0.77897512 1.8417265 ]
  [1.65849154 0.74640983 2.01065259]
  [1.30205716 0.53851687 1.6793864 ]]

 [[1.02483539 0.69452105 0.94906196]
  [1.11900687 0.74440623 1.17127183]
  [1.02057297 0.53319234 0.88871561]
  ...
  [0.99908574 0.34057256 0.41510893]
  [0.84424215 0.2998968  0.37949184]
  [0.87228096 0.39809851 0.42524684]]

 ...

 [[1.02483539 0.69452105 0.94906196]
  [1.11900687 0.74440623 1.17127183]
  [1.02057297 0.53319234 0.88871561]
  ...
  [1.45286713 2.8783833  1.58558192]
  [1.51350801 3.63023775 1.70230657]
  [1.18487081 3.09406966 1.64915344]]

 [[1.02483539 0.69452105 0.94906196]
  [1.11900687 0.74440623 1.17127183]


In [7]:
rets.shape

(100000, 20, 3)

In [8]:
np.savez_compressed('Datos/Transformados/montecarlo_data.npz', features=rets, target=payoff)

### Valor presente

Dados todos los timesteps:

In [9]:
#present_value (numpy.ndarray): present value of the payoff in every time step. Shape (num_sims, num_asian_dates-1)

time_array = np.linspace(0, initial_maturity, num_steps)

present_value = np.matmul(payoff.reshape(-1,1),np.exp(- risk_free_rate*(initial_maturity-time_array )).reshape(1,-1))

In [10]:
print(present_value)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.05920886 0.05936488 0.05952131 ... 0.06191782 0.06208098 0.06224456]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.33679166 0.33767912 0.33856892 ... 0.35220076 0.35312883 0.35405934]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
